Iris Example
=====

This example shows how to use QuickExperiment to define an experiment suite for quick iterations and high result reproducibility. It uses the sklearn Iris dataset and a simple Logistic Regression classifier.

First, we load the dataset, which is already preprocess into a numeric matrix and an array of labels

In [1]:
from sklearn import datasets
iris = datasets.load_iris()

We are going to create several splits of the dataset to use k-fold validations, so we define a subclass of the SampledDataset to organize this samples. 

In [5]:
# add parent directory to python path
import sys
sys.path.append('../')

from dataset import SimpleSampledDataset

We define now an experiment serie that we want to run